# Machine Learning Pipeline - Feature Engineering


1. Exploratory Data Analysis
2. **Feature Engineering**
    - Targeet transformation
    - Missing values
    - Outliers
    - Non-Gaussian distributed variables
    - Categorical variable: convert strings to numbers
    - Put the variables in a similar scale
3. Feature Selection
4. Model Trainng
5. Obtaining Predictions / Scoring



## 1. Data Loading

In [112]:
import pandas as pd
import numpy as np 

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [113]:
train = pd.read_csv('data/train.csv')
building = pd.read_csv('data/building_metadata.csv')
weather_train = pd.read_csv('data/weather_train.csv')

print('train:', train.shape)
print('building_metadata:', building.shape)
print('weather_train:', weather_train.shape)

data = train.merge(building, on = 'building_id', how = 'left')
data = data.merge(weather_train, on = ['site_id', 'timestamp'], how = 'left')

train: (20216100, 4)
building_metadata: (1449, 6)
weather_train: (139773, 9)


In [114]:
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['month'] = data['timestamp'].dt.month
data['day'] = data['timestamp'].dt.day
data['hour'] = data['timestamp'].dt.hour

In [115]:
X, Xtest, y, ytest = train_test_split(data.drop('meter_reading', axis = 1), data['meter_reading'], \
                                      test_size=0.15, random_state=0, shuffle = True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=0, shuffle = False)

In [116]:
X_train.head(2)

,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,month,day,hour
10850197,319,0,2016-07-17 22:00:00,3,Public services,13100,1929.0,NaN,32.8,NaN,17.8,0.0,1019.9,210.0,4.1,7,17,22
807753,309,0,2016-01-15 16:00:00,3,Entertainment/public assembly,2067,NaN,NaN,7.2,NaN,0.6,0.0,1011.8,170.0,4.1,1,15,16


In [117]:
ID_VARS = ['building_id', 'meter', 'site_id']
CATEGORICAL_BUILDING_VARS = ['primary_use']

TEMPORAL_VARS = ['hour','day', 'month', 'timestamp']

DISCRETE_BUILDING_VARS = ['floor_count', 'year_built']
NUMERICAL_BUILDING_VARS = ['square_feet']

DISCRETE_WEATHER_VARS = ['cloud_coverage']
NUMERICAL_WEATHER_VARS = ['air_temperature', 'dew_temperature', 'precip_depth_1_hr',
                         'sea_level_pressure', 'wind_direction','wind_speed'] 

## 2. Target

We can add to the target some positive value to make it no zero (not no remove from dataset). And then to make a log transformation. So for the prediction we will need to make a reverse procedure. 

In [118]:
y_train = np.log(y_train + 2)
y_val = np.log(y_val + 2)

## 3. Missing values

To engineer missing values in numerical variables, we will:

- add a binary missing indicator variable
- and then replace the missing values in the original variable by mode

    -- floor_count, year_built: mode according to primary_use
    
    -- weather variables: mode according to month

In [119]:
# variables with missing values

vars_with_na = [
    var for var in X.columns
    if X_train[var].isnull().mean() > 0 
]

X_train[vars_with_na].isnull().mean().sort_values(ascending = False)

floor_count           0.826493
year_built            0.599869
cloud_coverage        0.436381
precip_depth_1_hr     0.185462
wind_direction        0.071651
sea_level_pressure    0.060945
wind_speed            0.007094
dew_temperature       0.004941
air_temperature       0.004768
dtype: float64

In [120]:
# mode values for floor_count and year_built according to primary_use of buildings

# config_nan_floor_year = X_train.groupby(['primary_use'])['floor_count', 'year_built'].agg(pd.Series.mode)

# config_nan_floor_year.loc['Food sales and service']['floor_count'] = 1
# config_nan_floor_year.loc['Religious worship']['floor_count'] = 1
# config_nan_floor_year.loc['Services']['floor_count'] = 1

# config_nan_floor_year.loc['Services']['year_built'] = int(X_train['year_built'].mode())

# config_nan_floor_year.to_csv('config/config_nan_floor_year.csv')

In [121]:
# mode values for weather variables according to month

# config_nan_weather = X_train.groupby(['month'])[NUMERICAL_WEATHER_VARS+DISCRETE_WEATHER_VARS].mean()

# config_nan_weather['cloud_coverage'] = round(config_nan_weather['cloud_coverage'],0)

# for var in NUMERICAL_WEATHER_VARS:
#     if var in ['precip_depth_1_hr', 'wind_direction']:
#         config_nan_weather[var] = round(config_nan_weather[var], 0)
#     else:
#         config_nan_weather[var] = round(config_nan_weather[var], 1)
# config_nan_weather.to_csv('config/config_nan_weather.csv')

In [122]:
config_building = pd.read_csv('config/config_nan_floor_year.csv')
config_weather = pd.read_csv('config/config_nan_weather.csv')

for var in vars_with_na:
    
    # adding binary
    X_train[var + '_na'] = np.where(X_train[var].isnull(), 1, 0)
    X_val[var + '_na'] = np.where(X_val[var].isnull(), 1, 0)
        
    # mode according to primary_use (building variables)
    if var in ['floor_count', 'year_built']:
        for use in X_train['primary_use'].unique():
            X_train[var].fillna(X_train['primary_use'].map(dict(zip(config_building.primary_use, config_building[var]))), inplace = True)
            X_val[var].fillna(X_val['primary_use'].map(dict(zip(config_building.primary_use, config_building[var]))), inplace = True)

    else:
    # mode according to month (weather variables)
        for month in range(1,13):
            X_train[var].fillna(X_train['month'].map(dict(zip(config_weather.month, config_weather[var]))), inplace = True)
            X_val[var].fillna(X_val['month'].map(dict(zip(config_weather.month, config_weather[var]))), inplace = True)
    

In [123]:
print(X_train.isnull().sum().sort_values(ascending = False))
print(X_val.isnull().sum().sort_values(ascending = False))

building_id              0
wind_speed               0
wind_direction_na        0
sea_level_pressure_na    0
precip_depth_1_hr_na     0
dew_temperature_na       0
cloud_coverage_na        0
air_temperature_na       0
floor_count_na           0
year_built_na            0
hour                     0
day                      0
month                    0
wind_direction           0
meter                    0
sea_level_pressure       0
precip_depth_1_hr        0
dew_temperature          0
cloud_coverage           0
air_temperature          0
floor_count              0
year_built               0
square_feet              0
primary_use              0
site_id                  0
timestamp                0
wind_speed_na            0
dtype: int64
building_id              0
wind_speed               0
wind_direction_na        0
sea_level_pressure_na    0
precip_depth_1_hr_na     0
dew_temperature_na       0
cloud_coverage_na        0
air_temperature_na       0
floor_count_na           0
year_built_na  

In [124]:
NAN_INDICATOR_VARS = ['floor_count_na', 'year_built_na', 'air_temperature_na',
       'cloud_coverage_na', 'dew_temperature_na', 'precip_depth_1_hr_na',
       'sea_level_pressure_na', 'wind_direction_na', 'wind_speed_na']

## 4. Outliers

We will not remove any data for now.

Will be updated during the model developping process.

## 5. Non-Gaussian distributed variables

In [125]:
X_train['square_feet'] = np.log(X_train['square_feet'])
X_val['square_feet'] = np.log(X_val['square_feet'])

## 6. Categorical variable: convert strings to numbers


We will try 2 type of encoding: Label Encoding based on mean value of target for each category and One-Hot Encoding. During the training procewss we will see what works better.

One Hot Encoding can be good for tree-based algorithms. 

In [126]:
X_train.shape

(14606132, 27)

In [127]:
len(X_train['primary_use'].unique())

16

In [128]:
X_train['primary_use_1'] = X_train['primary_use']
X_val['primary_use_1'] = X_val['primary_use']

In [129]:
# one hot encoding with feature-engine
from feature_engine.encoding import OneHotEncoder as fe_OneHotEncoder

OneHotEnc = fe_OneHotEncoder(
    top_categories = None,
    variables = ['primary_use_1'],
    drop_last = True)

OneHotEnc.fit(X_train)

OneHotEncoder(drop_last=True, variables=['primary_use_1'])

In [130]:
X_train = OneHotEnc.transform(X_train)
X_val = OneHotEnc.transform(X_val)

In [131]:
X_train.head(2)

,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,...,primary_use_1_Lodging/residential,primary_use_1_Utility,primary_use_1_Other,primary_use_1_Healthcare,primary_use_1_Manufacturing/industrial,primary_use_1_Retail,primary_use_1_Food sales and service,primary_use_1_Parking,primary_use_1_Warehouse/storage,primary_use_1_Technology/science
10850197,319,0,2016-07-17 22:00:00,3,Public services,9.480368,1929.0,1.0,32.8,2.0,...,0,0,0,0,0,0,0,0,0,0
807753,309,0,2016-01-15 16:00:00,3,Entertainment/public assembly,7.633854,1976.0,1.0,7.2,2.0,...,0,0,0,0,0,0,0,0,0,0


In [132]:
X_train.columns

Index(['building_id', 'meter', 'timestamp', 'site_id', 'primary_use',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed', 'month', 'day',
       'hour', 'year_built_na', 'floor_count_na', 'air_temperature_na',
       'cloud_coverage_na', 'dew_temperature_na', 'precip_depth_1_hr_na',
       'sea_level_pressure_na', 'wind_direction_na', 'wind_speed_na',
       'primary_use_1_Public services',
       'primary_use_1_Entertainment/public assembly',
       'primary_use_1_Education', 'primary_use_1_Office',
       'primary_use_1_Services', 'primary_use_1_Lodging/residential',
       'primary_use_1_Utility', 'primary_use_1_Other',
       'primary_use_1_Healthcare', 'primary_use_1_Manufacturing/industrial',
       'primary_use_1_Retail', 'primary_use_1_Food sales and service',
       'primary_use_1_Parking', 'primary_use_1_Warehouse/storage',
       'pr

In [133]:
ONEHOTENC_VARS = [ 'primary_use_Public services',
       'primary_use_Entertainment/public assembly', 'primary_use_Education',
       'primary_use_Office', 'primary_use_Services',
       'primary_use_Lodging/residential', 'primary_use_Utility',
       'primary_use_Other', 'primary_use_Healthcare',
       'primary_use_Manufacturing/industrial', 'primary_use_Retail',
       'primary_use_Food sales and service', 'primary_use_Parking',
       'primary_use_Warehouse/storage', 'primary_use_Technology/science']

In [134]:
# # label encoding of a categorical variable based on mean value of target for each category

# tmp = pd.concat([X_train, y_train], axis=1)

# ordered_labels = tmp.groupby('primary_use')['meter_reading'].mean().sort_values().index

# primary_label = {k: i for i, k in enumerate(ordered_labels, 0)}

# import json

# with open('config/primary_label_encoding.json', 'w') as fp:
#     json.dump(primary_label, fp)

In [135]:
primary_label_dict = json.load(open('config/primary_label_encoding.json'))
X_train['primary_use'] = X_train['primary_use'].map(primary_label_dict)
X_val['primary_use'] = X_val['primary_use'].map(primary_label_dict)

## 7. Standartization

We scale features to the minimum and maximum values.

In [136]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [137]:
X_train.drop('timestamp', axis = 1, inplace = True)
X_val.drop('timestamp', axis = 1, inplace = True)

In [138]:
scaler.fit(X_train)

MinMaxScaler()

In [139]:
X_train = pd.DataFrame(
    scaler.transform(X_train),
    columns = X_train.columns)

X_val = pd.DataFrame(
    scaler.transform(X_val),
    columns = X_val.columns)

In [140]:
X_train.head()

,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,...,primary_use_1_Lodging/residential,primary_use_1_Utility,primary_use_1_Other,primary_use_1_Healthcare,primary_use_1_Manufacturing/industrial,primary_use_1_Retail,primary_use_1_Food sales and service,primary_use_1_Parking,primary_use_1_Warehouse/storage,primary_use_1_Technology/science
0,0.220304,0.000000,0.200000,0.533333,0.477186,0.247863,0.00,0.810775,0.222222,0.864157,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.213398,0.000000,0.200000,0.666667,0.247421,0.649573,0.00,0.474376,0.222222,0.582651,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.814917,0.666667,0.866667,1.000000,0.723956,0.649573,0.04,0.547963,0.222222,0.700491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.794890,0.000000,0.866667,0.800000,0.852552,0.649573,0.00,0.605782,0.222222,0.754501,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.616022,0.000000,0.600000,0.933333,0.925804,0.649573,0.00,0.634691,0.000000,0.608838,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
X_train.isnull().sum()

building_id                                    0
meter                                          0
site_id                                        0
primary_use                                    0
square_feet                                    0
year_built                                     0
floor_count                                    0
air_temperature                                0
cloud_coverage                                 0
dew_temperature                                0
precip_depth_1_hr                              0
sea_level_pressure                             0
wind_direction                                 0
wind_speed                                     0
month                                          0
day                                            0
hour                                           0
year_built_na                                  0
floor_count_na                                 0
air_temperature_na                             0
cloud_coverage_na   

In [143]:
X_train.describe()

,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,...,primary_use_1_Lodging/residential,primary_use_1_Utility,primary_use_1_Other,primary_use_1_Healthcare,primary_use_1_Manufacturing/industrial,primary_use_1_Retail,primary_use_1_Food sales and service,primary_use_1_Parking,primary_use_1_Warehouse/storage,primary_use_1_Technology/science
count,1.460613e+07,1.460613e+07,1.460613e+07,1.460613e+07,1.460613e+07,1.460613e+07,1.460613e+07,1.460613e+07,1.460613e+07,1.460613e+07,...,1.460613e+07,1.460613e+07,1.460613e+07,1.460613e+07,1.460613e+07,1.460613e+07,1.460613e+07,1.460613e+07,1.460613e+07,1.460613e+07
mean,5.519540e-01,2.208539e-01,5.327806e-01,7.770232e-01,6.711671e-01,6.438887e-01,6.333316e-02,5.898577e-01,2.119624e-01,6.996343e-01,...,1.061292e-01,2.776574e-03,1.199845e-02,1.966701e-02,6.231492e-03,5.580464e-03,5.644205e-03,1.059254e-02,5.514054e-03,3.844755e-03
std,2.948413e-01,3.103717e-01,3.399445e-01,2.355059e-01,1.468890e-01,1.865866e-01,1.076912e-01,1.437446e-01,2.014091e-01,1.664393e-01,...,3.080029e-01,5.262000e-02,1.088783e-01,1.388532e-01,7.869346e-02,7.449378e-02,7.491561e-02,1.023735e-01,7.405167e-02,6.188678e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.714088e-01,0.000000e+00,2.000000e-01,6.666667e-01,5.903515e-01,6.495726e-01,0.000000e+00,4.901445e-01,0.000000e+00,5.728314e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.180939e-01,0.000000e+00,6.000000e-01,8.000000e-01,6.904438e-01,6.495726e-01,4.000000e-02,5.992116e-01,2.222222e-01,7.184943e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,8.142265e-01,3.333333e-01,8.666667e-01,1.000000e+00,7.711778e-01,6.495726e-01,4.000000e-02,6.964520e-01,2.222222e-01,8.363339e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [141]:
# save scaler

import joblib

joblib.dump(scaler, 'minmax_scaler.joblib')

['minmax_scaler.joblib']

In [145]:
# save data sets

X_train.to_csv('data/preprocessed/xtrain.csv')
X_val.to_csv('data/preprocessed/xval.csv')

y_train.to_csv('data/preprocessed/ytrain.csv')
y_val.to_csv('data/preprocessed/yval.csv')